<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/%E8%BC%95%E6%B2%B9%E9%A0%90%E6%B8%AC%26%E7%89%B9%E5%BE%B5%E5%B7%A5%E7%A8%8B%26%E4%BA%A4%E4%BA%92%E9%A0%85%26Lasso%26SVR%26%E8%AA%BF%E6%95%B4%E8%B6%85%E5%8F%83%E6%95%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 訓練資料

In [2]:
import pandas as pd

train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/輕油實驗數據(保留大部份欄位,三個基本規則清洗過).csv')
train.rename(columns={'N+A in Feed':'NA'},inplace=True)

x_columns = ['T10','T50','T90','NA']
y_columns = ['C5NP', 'C5IP', 'C5N', 'C6NP',
       'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP',
       'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N',
       'C10A']
train = train[x_columns+y_columns]
print(train.shape)
train.head()

(4521, 27)


,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,100.5,119.2,146.5,31.978,0.272,0.132,0.059,5.293,2.570,2.819,0.494,10.395,8.070,6.411,2.917,9.138,9.649,4.810,5.373,6.405,9.759,4.590,3.661,0.875,5.257,0.525,0.319
1,99.6,117.9,145.5,31.568,0.297,0.151,0.062,5.089,2.531,2.804,0.499,10.074,7.958,6.395,2.894,8.970,9.548,4.753,5.443,6.324,9.899,4.301,2.995,0.881,5.591,1.119,0.303
2,100.0,118.8,145.6,31.344,0.262,0.126,0.061,5.107,2.571,2.771,0.492,10.069,7.913,6.378,2.890,9.006,9.591,4.778,5.468,6.360,9.983,4.274,2.979,0.865,5.641,0.964,0.289
3,100.4,118.6,142.9,31.453,0.224,0.105,0.060,4.894,2.497,2.650,0.469,10.015,7.685,6.376,2.866,9.133,9.708,4.889,5.510,6.444,10.182,4.420,2.964,0.830,5.637,0.968,0.281
4,100.4,118.1,142.2,32.190,0.243,0.117,0.061,4.946,2.503,2.695,0.477,10.053,7.765,6.394,2.877,9.101,9.676,4.855,5.500,6.416,10.115,4.347,3.725,0.835,4.823,0.969,0.290


# 特徵工程(訓練資料)

In [0]:
def 特徵工程(df):
  df['T90-T50'] = df['T90']-df['T50']
  df['T50-T10'] = df['T50']-df['T10']
  return df

In [4]:
train = 特徵工程(train)
train.head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,T90-T50,T50-T10
0,100.5,119.2,146.5,31.978,0.272,0.132,0.059,5.293,2.570,2.819,0.494,10.395,8.070,6.411,2.917,9.138,9.649,4.810,5.373,6.405,9.759,4.590,3.661,0.875,5.257,0.525,0.319,27.3,18.7
1,99.6,117.9,145.5,31.568,0.297,0.151,0.062,5.089,2.531,2.804,0.499,10.074,7.958,6.395,2.894,8.970,9.548,4.753,5.443,6.324,9.899,4.301,2.995,0.881,5.591,1.119,0.303,27.6,18.3
2,100.0,118.8,145.6,31.344,0.262,0.126,0.061,5.107,2.571,2.771,0.492,10.069,7.913,6.378,2.890,9.006,9.591,4.778,5.468,6.360,9.983,4.274,2.979,0.865,5.641,0.964,0.289,26.8,18.8
3,100.4,118.6,142.9,31.453,0.224,0.105,0.060,4.894,2.497,2.650,0.469,10.015,7.685,6.376,2.866,9.133,9.708,4.889,5.510,6.444,10.182,4.420,2.964,0.830,5.637,0.968,0.281,24.3,18.2
4,100.4,118.1,142.2,32.190,0.243,0.117,0.061,4.946,2.503,2.695,0.477,10.053,7.765,6.394,2.877,9.101,9.676,4.855,5.500,6.416,10.115,4.347,3.725,0.835,4.823,0.969,0.290,24.1,17.7


# 加入新特徵至x_columns

In [5]:
x_columns = x_columns+['T90-T50','T50-T10']
print(x_columns)

['T10', 'T50', 'T90', 'NA', 'T90-T50', 'T50-T10']


# Lasso特徵篩選器

In [0]:
from sklearn.base import TransformerMixin
from sklearn.linear_model import Lasso
import numpy as np

class LassoTransformer(TransformerMixin):  
  def __init__(self):#初始化首先建立一個Lasso model
    self.model = Lasso()
  
  def set_params(self,alpha):#設置alpha參數會影響選擇變數多寡
    self.model.set_params(alpha=alpha)
  
  def fit(self,X,y):#對資料做訓練得到coef_
    self.model.fit(X,y)
    return self
  
  def transform(self,X):#呼叫此方法可過濾掉冗餘特徵
    SelectColumn = (abs(self.model.coef_)!=0)
    return X[:,SelectColumn]

功能測試

In [7]:
y_name = 'C5NP'

X_train = np.array(train[x_columns])
y_train = np.array(train[y_name])

LassoTR = LassoTransformer()#建立LassoTR
LassoTR.set_params(alpha=0.5)#設定alpha
LassoTR.fit(X_train,y_train)#fit

print(X_train.shape)#LassoTR轉換前
print("==After Lasso==")
print(LassoTR.transform(X_train).shape)#LassoTR轉換後

(4521, 6)
==After Lasso==
(4521, 2)


# 交互項特徵產生器

In [0]:
from sklearn.preprocessing import PolynomialFeatures

class featuresGenerator(TransformerMixin):
  def __init__(self):#初始化建立一個交互項產生器,只兩兩相乘,不包括bias
    self.model = PolynomialFeatures(2,interaction_only=True,include_bias=False)
  
  def fit(self,X,y):
    return self
  
  def transform(self,X): 
    return  self.model.fit_transform(X) 

功能測試

In [9]:
print(X_train.shape)
f_tr = featuresGenerator()
print('==after features generator==')
print(f_tr.transform(X_train).shape)

(4521, 6)
==after features generator==
(4521, 21)


# pipeline流程 input -> StandardScaler -> featuresGenerator -> Lasso -> svr 

In [0]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

scaler = StandardScaler() #特徵縮放
f_tr = featuresGenerator() #產生交互項
LassoTR = LassoTransformer() #篩選因子
svr = svm.SVR()#svr回歸模型
xgb = XGBRegressor()

pipeline = Pipeline([('scaler',scaler),
                     ('f_tr',f_tr),
                     ('LassoTR',LassoTR),
                     ('svr',svr)])

# 評分工具

In [0]:
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def mape(y_true, y_pred,dropinf=True):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ape = np.abs((y_true - y_pred) / y_true)
    if dropinf == True:
      ape = ape[np.isfinite(ape)]
    return np.mean(ape) * 100

def metric(y_true,y_pred):
  R2 = r2_score(y_true,y_pred)
  RMSE = rmse(y_true,y_pred)
  MAPE = mape(y_true,y_pred)
  return {'R2':R2,'RMSE':RMSE,'MAPE':MAPE}

# train test資料準備

In [12]:
#訓練資料
X_train = train[x_columns]
y_train = train[y_columns]
print(X_train.shape)
print(y_train.shape)

(4521, 6)
(4521, 23)


In [13]:
#讀取測試資料170筆
test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/test_without_outlier.csv')
#rename columns
test.columns = train.columns[:-2]
#特徵工程
test = 特徵工程(test)
#測試資料
X_test = test[x_columns]
y_test = test[y_columns]
print(X_test.shape)
print(y_test.shape)

(170, 6)
(170, 23)


#調整超參數

In [14]:
import warnings 
warnings.simplefilter('ignore')
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm_notebook as tqdm

table = pd.DataFrame()

#人工定義超參數
ALPHA = 0.04
C = 1
GAMMA = 'scale'

#對23個y建模
for y_name in tqdm(y_columns):
  model = pipeline.set_params(LassoTR__alpha = ALPHA,
                              svr__C = C,
                              svr__gamma=GAMMA)
  model.fit(X_train,y_train[y_name])#訓練
  y_pred = model.predict(X_test)#預測
  table = table.append(pd.DataFrame(metric(y_test[y_name],y_pred),index=[y_name]))#append結果至"table"

#table平均
table.loc['AVG',:] = table.mean(axis=0)
table

,R2,RMSE,MAPE
C5NP,0.884713,0.290165,27.219357
C5IP,0.738117,0.261873,41.651664
C5N,0.785770,0.063363,225.162305
C6NP,0.088396,0.386282,6.601181
C6IP,0.866548,0.381845,9.492352
C6N,0.879682,0.574429,5.500986
C6A,0.533583,0.116681,17.532331
C7NP,0.980630,0.306088,2.941694
C7IP,0.859847,0.413681,4.421077
C7N,0.980959,0.575045,3.406385


# 檢查目前最大RMSE

In [15]:
table[table.RMSE==table.RMSE.max()]

,R2,RMSE,MAPE
C7N,0.980959,0.575045,3.406385


# 結論:已達驗收標準,無清洗資料,平均RMSE:0.315,個別最大RMSE不超過0.6

# 嘗試針對RMSE大於0.50以上的加入集成學習,另寫一份文件

In [16]:
High_RMSE_index = table[table['RMSE']>0.50].index.tolist()
High_RMSE_index

['C6N', 'C7N', 'C8N', 'C8A']